<a href="https://colab.research.google.com/github/quotation3/TIL/blob/master/%ED%95%9C%EA%B5%AD%EC%9D%80%ED%96%89%EB%85%BC%EB%AC%B8%EA%B5%AC%ED%98%84_%EA%B8%B0%EC%A4%80%EA%B8%88%EB%A6%AC%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd

In [ ]:
policy_rates = []
dates = []

res = requests.get("https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643")
bs = BeautifulSoup(res.text, "html.parser")
# print(bs.text)
content = bs.select('table tbody tr')
for i in content:
    year=i.select('td')[0].text
    month_day=i.select('td')[1].text.replace('월','').replace('일','').replace(' ','')
    policy_rates.append(i.select('td')[2].text)
    date = datetime.strptime(year+month_day, '%Y%m%d').strftime('%Y-%m-%d')
    dates.append(date)

In [73]:
data = np.array([dates,policy_rates])
df_policy_rates = pd.DataFrame(data.T, columns=['date','policy_rate'])
# df_policy_rates

In [ ]:
# 중간중간 빈날짜를 채워줘야함.
from datetime import datetime, timedelta
# start_date부터 end_date까지 날짜 리스트 생성
start_date = datetime.strptime('20050101', '%Y%m%d')
end_date = datetime.strptime('20200722', '%Y%m%d')

str_date_list = []
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): # start_date가 end_date랑 같아지기 전까지 1씩 늘어나면서 리스트에 추가
    str_date_list.append(start_date.strftime('%Y-%m-%d'))
    start_date += timedelta(days=1)
str_date_list.append(end_date.strftime('%Y-%m-%d')) # end_date 포함안됐으므로 따로 추가
    
date_list=np.array([str_date_list])
all_date  = pd.DataFrame(date_list.T, columns=['date'])
# print(all_date)

In [74]:
df = pd.merge(df_policy_rates,all_date, on='date', how='outer') # 기존크롤링데이터와 전체날짜를 date기준으로 outer 조인
df_policyrate = df.sort_values(by='date' ,ascending=False).fillna(method='bfill') # 내림차순으로 정렬후, 결측치는 가장 최근 과거 값으로 대입
# df_policyrate.head(100)

In [75]:
df_policyrate.drop(df_policyrate.tail(12).index,inplace=True)   #2005년 이전 자료는 삭제
df_policyrate.tail(10)

,date,policy_rate
54,2005-01-10,3.25
53,2005-01-09,3.25
52,2005-01-08,3.25
51,2005-01-07,3.25
50,2005-01-06,3.25
49,2005-01-05,3.25
48,2005-01-04,3.25
47,2005-01-03,3.25
46,2005-01-02,3.25
45,2005-01-01,3.25


In [ ]:
df_policyrate.to_csv('policy_rate.csv', index=False, encoding="utf-8")